In [1]:
import json
import os
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
import keras
from keras.layers import Dropout
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

# Restart the environment after the installation and before the import
from mmpose.core.visualization import imshow_keypoints

***Prepare Data***

In [2]:
# from google.colab import files
# from google.colab import drive
# import json

# drive.mount("/content/drive")

In [17]:
to_compute = "full"

In [18]:
body_poses = np.array([], dtype=np.float64).reshape(0, 64)
labels = []

FOLDER_PATH = '../posture_data/'+to_compute+'/'

# Loop through the annotation files
for file_label in np.array(os.listdir(FOLDER_PATH)):
  # Read the file content
  with open(FOLDER_PATH + file_label) as f:
    file_annotations = json.load(f)
  print(file_label)

  # Loop through the images and save the data
  for image in file_annotations.values():
    body_poses = np.vstack([ body_poses, np.hstack((
          np.array(
              list(image['person_1'].values())), 
          np.array(
              list(image['person_2'].values()))
          )).flatten() ])
    labels.append(int(file_label[0]))

# Remove 1 to the labels so not to have one more element in the hot encoding
labels = [x - 1 for x in labels]

1_full_bb.json
6_full_bb.json
2_full_bb.json
5_full_bb.json
4_full_bb.json
3_full_bb.json


In [19]:
list(set(labels))

[0, 1, 2, 3, 4, 5]

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(body_poses)
body_poses = scaler.transform(body_poses)

# Transform to one-hot encoding
y = to_categorical(labels).astype(int)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(body_poses, y, test_size=0.05)

In [22]:
# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder()
# encoder.fit(np.array(y_train).reshape(-1, 1))
# y_train = encoder.transform(np.array(y_train).reshape(-1, 1)).toarray()
# y_test = encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()

## **Preparing the neural network**

Load and Split dataset

In [23]:
# #shuffle the dataset
# df = df.sample(frac=1).reset_index(drop=True)

# # split X & Y
# Y = df['label']
# X = df.drop(['label'], axis=1)

# print(X.shape)
# print(Y.shape)

# # convert to numpy arrays
# X = np.array(X)

Dummy Encode Labels

In [24]:
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
# # one hot enocde the integers
# dummy_y = np_utils.to_categorical(encoded_Y)

# print(encoded_Y)
# print(dummy_y)

Model

In [25]:
#model
model = Sequential()
# model.add(Dropout(0.1, input_shape=(64,)))
model.add(Dense(64, input_shape=(x_train.shape[1],), activation='relu')) # input shape is (features,)
model.add(Dense(units = 128, 
                kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                # kernel_regularizer='l2',
                activation='relu'))
model.add(Dense(units = 64,
                activation='relu'))
# model.add(Dense(units = 32,
#                 activation='relu'))
model.add(Dense(units = 16,
                activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

# compile the model
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
 dense_9 (Dense)             (None, 6)                 102       
                                                                 
Total params: 21,878
Trainable params: 21,878
Non-trainable params: 0
_________________________________________________________________


Simple MLP

In [27]:
# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 5 consecutive epochs.  
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=20, 
                                   restore_best_weights=True) # important - otherwise you just return the last weigths...

# update our model fit call
history = model.fit(x_train,
                    y_train,
                    callbacks=[es],
                    epochs=100, # How many?
                    batch_size=25,
                    shuffle=True,
                    # validation_split=0.2,
                    validation_data=(x_test, y_test),
                    verbose=1)

Epoch 1/100
2313/2313 [==============================] - 1s 635us/step - loss: 1.7919 - accuracy: 0.1659 - val_loss: 1.7923 - val_accuracy: 0.1715
Epoch 2/100
2313/2313 [==============================] - 1s 638us/step - loss: 1.7919 - accuracy: 0.1643 - val_loss: 1.7922 - val_accuracy: 0.1718
Epoch 3/100
2313/2313 [==============================] - 2s 701us/step - loss: 1.7918 - accuracy: 0.1672 - val_loss: 1.7917 - val_accuracy: 0.1616
Epoch 4/100
2313/2313 [==============================] - 1s 626us/step - loss: 1.7918 - accuracy: 0.1686 - val_loss: 1.7924 - val_accuracy: 0.1718
Epoch 5/100
2313/2313 [==============================] - 2s 656us/step - loss: 1.7918 - accuracy: 0.1690 - val_loss: 1.7915 - val_accuracy: 0.1718
Epoch 6/100
2313/2313 [==============================] - 1s 623us/step - loss: 1.7919 - accuracy: 0.1688 - val_loss: 1.7927 - val_accuracy: 0.1491
Epoch 7/100
2313/2313 [==============================] - 1s 616us/step - loss: 1.7919 - accuracy: 0.1687 - val_loss: 1

In [17]:
model.save('../models/nn/'+to_compute)

INFO:tensorflow:Assets written to: ../models/nn/five_hundred/assets


In [28]:
model.evaluate(x_test, y_test)

96/96 [==============================] - 0s 376us/step - loss: 1.7915 - accuracy: 0.1718


[1.791486382484436, 0.17181339859962463]


Evaluation

In [ ]:
history_dict = history.history

# learning curve
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

# plot
# "r" is for "solid red line"
plt.plot(epochs, acc, 'r', label='Training accuracy')
# b is for "solid blue line"
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(x_test) 
matrix = confusion_matrix(y_test.argmax(axis=1), preds.argmax(axis=1))
print("Confusion matrix")
print(matrix)
print('')
# more detail on how well things were predicted
print(classification_report(y_test.argmax(axis=1), preds.argmax(axis=1)))